In [9]:
text = """# Study Buddy - LLM Agent & Retrieval (Week 5 Assignment)

## Problem Statement
Students often have long lecture notes and struggle to quickly find answers. This project builds a local Document Q&A assistant that reads your notes and answers questions. It also includes a simple calculator tool.

## Solution Architecture
- Retrieval: SentenceTransformers (all-MiniLM-L6-v2)
- Generation: HuggingFace flan-t5-base
- Tool integration: Local safe calculator
- Implemented in Colab notebook

## Setup Instructions
1. Upload notebook into Colab
2. Upload `report_ai_project.txt` file
3. Run cells top-to-bottom
4. Example queries:
   calculate("23*45")
   ask_ai("Explain gradient descent")

## Demo Video
Record 5–7 min video as per assignment guidelines

## Future Improvements
- Stronger models
- Web UI
- Multi-document indexing
"""
with open("README.md", "w") as f:
    f.write(text)


In [1]:
!pip install sentence-transformers transformers python-docx

import torch
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import docx
import ast


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 5.8 MB/s eta 0:00:00


In [2]:
from google.colab import files

uploaded = files.upload()
filename = list(uploaded.keys())[0]

if filename.endswith(".docx"):
    doc = docx.Document(filename)
    text_data = "\n".join([p.text for p in doc.paragraphs])
else:
    with open(filename, "r", encoding="utf-8") as f:
        text_data = f.read()

print("File Loaded")


Saving report_ai_project.txt to report_ai_project.txt
✅ File Loaded


In [3]:
def split_into_chunks(text, max_len=200):
    words = text.split()
    chunks, current = [], []
    count = 0
    for w in words:
        current.append(w)
        count += 1
        if count >= max_len:
            chunks.append(" ".join(current))
            current, count = [], 0
    if current:
        chunks.append(" ".join(current))
    return chunks

chunks = split_into_chunks(text_data)
print("Total Chunks:", len(chunks))


Total Chunks: 3


In [4]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")
chunk_embeddings = embedder.encode(chunks, convert_to_tensor=True)

class RetrievalAgent:
    def __init__(self, chunks):
        self.chunks = chunks
        self.embeddings = embedder.encode(chunks, convert_to_tensor=True)

    def process_query(self, query):
        query_embedding = embedder.encode(query, convert_to_tensor=True)
        scores = util.pytorch_cos_sim(query_embedding, self.embeddings)[0]
        best_idx = int(torch.argmax(scores))
        context = self.chunks[best_idx]

        generator = pipeline("text-generation", model="gpt2")
        prompt = f"""
You are an AI tutor. Use this context to answer clearly and in simple words.

Context:
{context}

Question: {query}
Answer:
"""
        result = generator(prompt, max_new_tokens=120)[0]['generated_text']
        return result

agent = RetrievalAgent(chunks)
print(" Agent Ready")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

 Agent Ready


In [5]:
def calculate(expr):
    node = ast.parse(expr, mode='eval')

    def eval_node(node):
        if isinstance(node, ast.Constant):
            return node.value
        elif isinstance(node, ast.BinOp):
            left, right = eval_node(node.left), eval_node(node.right)
            if isinstance(node.op, ast.Add): return left + right
            if isinstance(node.op, ast.Sub): return left - right
            if isinstance(node.op, ast.Mult): return left * right
            if isinstance(node.op, ast.Div): return left / right
        else:
            raise ValueError("Unsupported operation")
    return eval_node(node.body)


In [6]:
def ask_ai(query):
    return agent.process_query(query)


In [7]:
print("Calculator:", calculate("150/6"))
print("AI Answer:", ask_ai("Explain gradient descent"))
print("AI Answer:", ask_ai("List applications of AI"))
print("AI Answer:", ask_ai("What are the strengths and weaknesses of AI?"))
print("AI Answer:", ask_ai("Give me a short summary of the report"))

Calculator: 25.0


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

AI Answer: 
You are an AI tutor. Use this context to answer clearly and in simple words.

Context:
Title: Artificial Intelligence and Its Applications Introduction: Artificial Intelligence (AI) is a branch of computer science that aims to create machines that can perform tasks requiring human intelligence. It includes techniques such as machine learning, natural language processing, computer vision, and robotics. AI systems can analyze data, learn patterns, and make decisions with little or no human intervention. History of AI: The concept of intelligent machines has existed for centuries, but modern AI research began in the 1950s. Alan Turing, known as the father of computer science, proposed the idea of machines simulating human intelligence. Early AI programs were rule-based, but they were limited in handling complex real-world problems. Applications of AI: 1. Healthcare – AI helps in diagnosing diseases, analyzing medical images, and discovering new drugs. 2. Finance – Fraud detect

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


AI Answer: 
You are an AI tutor. Use this context to answer clearly and in simple words.

Context:
is not just a technology—it is a transformational force. With its rapid growth, AI will continue to reshape our world, offering new opportunities and challenges. References: - Russell, S., & Norvig, P. (2020). Artificial Intelligence: A Modern Approach. - Goodfellow, I., Bengio, Y., & Courville, A. (2016). Deep Learning. - Various online research articles and case studies.

Question: What are the strengths and weaknesses of AI?
Answer:

AI is not just a technology—it is a transformational force. With its rapid growth, AI will continue to reshape our world, offering new opportunities and challenges. References: - Russell, S., & Norvig, P. (2020). Artificial Intelligence: A Modern Approach. - Goodfellow, I., Bengio, Y., & Courville, A. (2016). Deep Learning. - Various online research articles and case studies.

What are the strengths and weaknesses of AI?

There is a broad consensus among l

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


AI Answer: 
You are an AI tutor. Use this context to answer clearly and in simple words.

Context:
Title: Artificial Intelligence and Its Applications Introduction: Artificial Intelligence (AI) is a branch of computer science that aims to create machines that can perform tasks requiring human intelligence. It includes techniques such as machine learning, natural language processing, computer vision, and robotics. AI systems can analyze data, learn patterns, and make decisions with little or no human intervention. History of AI: The concept of intelligent machines has existed for centuries, but modern AI research began in the 1950s. Alan Turing, known as the father of computer science, proposed the idea of machines simulating human intelligence. Early AI programs were rule-based, but they were limited in handling complex real-world problems. Applications of AI: 1. Healthcare – AI helps in diagnosing diseases, analyzing medical images, and discovering new drugs. 2. Finance – Fraud detect

In [8]:
!pip freeze > requirements.txt

In [10]:
test_code = """from src.utils_calc import calculate

def test_calc():
    assert calculate("2+3") == 5
    assert calculate("10/2") == 5
"""
with open("test_calculator.py", "w") as f:
    f.write(test_code)


In [11]:
problem_text = """# Problem Analysis

## Problem Identification
Students often struggle with managing and retrieving information from large sets of lecture notes, assignments, and research materials. Searching manually consumes time and reduces productivity, especially when deadlines are close.

## Target Users
- University students preparing for exams or assignments
- Researchers who need quick fact-checking from long documents
- Professionals handling long technical documents

## Current Challenges
1. Manual searching is time-consuming and inefficient.
2. Existing solutions (like Ctrl+F) are limited and only match keywords, not context.
3. Lack of integration with productivity tools such as calculators or simple automation.
4. Students want user-friendly systems without paid APIs (like OpenAI).

## Proposed Solution
A **Document Q&A Assistant** built with LlamaIndex/HuggingFace:
- Reads uploaded `.txt` lecture notes or reports.
- Lets users ask questions in natural language.
- Retrieves the most relevant chunk of text using embeddings.
- Generates a simple, clear answer using a local HuggingFace model (no paid API).
- Provides an integrated calculator for quick math queries.

## Success Criteria
-  Accurate answers from lecture notes.
-  Safe local calculator tool for math.
-  Easy to run on Google Colab without paid APIs.
-  Structured repository with documentation and demo.
"""

with open("problem-analysis.md", "w") as f:
    f.write(problem_text)
print(" problem-analysis.md created")


 problem-analysis.md created


In [12]:
architecture_text = """# Technical Architecture

## Overview
The system is designed as a lightweight, retrieval-based assistant that works fully on Google Colab without requiring paid APIs. It integrates:
- Document retrieval (vector embeddings + similarity search)
- Local text generation model
- A simple calculator tool
- Clear documentation for easy reproducibility

## Components

### 1. Document Processing
- Input: `.txt` or `.docx` file uploaded by the user.
- Tool: `SentenceTransformers (all-MiniLM-L6-v2)` for embedding text.
- Step: File split into chunks → embeddings stored → vector search applied.

### 2. Retrieval System
- Uses cosine similarity to match user query with the most relevant chunk.
- Returns top-k context to feed into the language model.

### 3. Language Model
- Model: `google/flan-t5-base` from HuggingFace.
- Task: Generates concise answers using context + query.
- Runs on CPU in Colab (works offline with HuggingFace hub).

### 4. Tool Integration (Calculator)
- Built using Python `ast` for safe math evaluation.
- Example:
  - Input: `23*45`
  - Output: `1035`

### 5. User Interaction
- Functions:
  - `calculate(expression)` → math results
  - `ask_ai(query)` → answers from uploaded document
- Output shown with `print()` in Colab.

## Flow Diagram (Textual)
1. User uploads notes →
2. Notes chunked + embedded →
3. User asks query →
4. Retriever finds best chunk →
5. HuggingFace model generates answer →
6. Answer displayed →
7. If query is math → calculator handles.

## Error Handling
- Invalid math expression → returns `"Error: Invalid expression"`
- Missing file → shows `"Please upload a document"`
- Model loading errors → fallback to smaller model.

## Future Improvements
- Add web-based UI (Gradio/Streamlit).
- Multi-file indexing.
- Integration with APIs like web search.
"""

with open("architecture.md", "w") as f:
    f.write(architecture_text)
print(" architecture.md created")


 architecture.md created
